In [1]:
from pandas import read_csv, unique

import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from scipy.stats import mode

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow import stack
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling1D, BatchNormalization, MaxPool1D, Reshape, Activation
from keras.layers import Conv1D, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [4]:
# 데이터 로드
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
data = pd.read_csv(url, names=columns)

def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

data["class"].replace(regex = True, inplace = True, to_replace = r';', value = r'')
    # ... and then this column must be transformed to float explicitly
data["class"] = data["class"].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss will show up as NAN
data.dropna(axis=0, how='any', inplace=True)
data.head()

,buying,maint,doors,persons,lug_boot,safety,class


In [8]:
label_encode = LabelEncoder()
data['classEncode'] = label_encode.fit_transform(data['class'].values.ravel())
data

,buying,maint,doors,persons,lug_boot,safety,class,classEncode


In [14]:
'''
Using linear interpolation we can overcome the problem of data that is lost during the collection process which is indicated by NaN. 
It will fill the missing values. Although in this dataset there is only one NaN value still I have implemented it for your convenience.
'''

'\nUsing linear interpolation we can overcome the problem of data that is lost during the collection process which is indicated by NaN. \nIt will fill the missing values. Although in this dataset there is only one NaN value still I have implemented it for your convenience.\n'

In [15]:
interpolation_fn = interp1d(data['activityEncode'] ,data['class'], kind='linear')
null_list = data[data['class'].isnull()].index.tolist()
for i in null_list:
    y = data['activityEncode'][i]
    value = interpolation_fn(y)
    data['class']=data['class'].fillna(value)
    print(value)

# Data Split

In [16]:
## train split users between 1 and 27, test split users between 28 and 33
df_test = data[data['user-id'] > 27]
df_train = data[data['user-id'] <= 27]

# Normalization

In [17]:
df_train['X'] = (df_train['X']-df_train['X'].min())/(df_train['X'].max()-df_train['X'].min())
df_train['Y'] = (df_train['Y']-df_train['Y'].min())/(df_train['Y'].max()-df_train['Y'].min())
df_train['Z'] = (df_train['Z']-df_train['Z'].min())/(df_train['Z'].max()-df_train['Z'].min())
df_train

,user-id,activity,timestamp,X,Y,Z,activityEncode
8489,17,Walking,57869902574000,0.510163,0.741908,0.488932,5
8490,17,Walking,57870012316000,0.495703,0.751526,0.487895,5
8491,17,Walking,57870122301000,0.489850,0.742938,0.484785,5
8492,17,Walking,57870222246000,0.499490,0.748778,0.489623,5
8493,17,Walking,57870332292000,0.499490,0.750496,0.486858,5
...,...,...,...,...,...,...,...
1098199,19,Sitting,131623331483000,0.723205,0.454981,0.545293,2
1098200,19,Sitting,131623371431000,0.724216,0.457755,0.546308,2
1098201,19,Sitting,131623411592000,0.725228,0.459773,0.545293,2
1098202,19,Sitting,131623491487000,0.723205,0.457755,0.546308,2


# Segmentation

Reshaping of dataframe is needed for it to be fed into our model. For this, we will create a function called segments that will take in the dataframe, the label names, and the length of each record which is set to 80 steps. This function will also perform separation of features (X-acceleration, Y-acceleration, Z-acceleration) and labels in x_train and y_train respectively.

In [19]:
def segments(df, time_steps, step, label_name):
    N_FEATURES = 3
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['X'].values[i:i+time_steps]
        ys = df['Y'].values[i:i+time_steps]
        zs = df['Z'].values[i:i+time_steps]
        
        label = mode(df[label_name][i:i+time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)
    
    reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels

TIME_PERIOD = 80
STEP_DISTANCE = 40
LABEL = 'activityEncode'
x_train, y_train = segments(df_train, TIME_PERIOD, STEP_DISTANCE, LABEL)

In [20]:
print('x_train shape:', x_train.shape)
print('Training samples:', x_train.shape[0])
print('y_train shape:', y_train.shape)

x_train shape: (20334, 80, 3)
Training samples: 20334
y_train shape: (20334,)


In [21]:
## Input and Output Dimensions
time_period, sensors = x_train.shape[1], x_train.shape[2]
num_classes = label_encode.classes_.size
print(list(label_encode.classes_))

['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']


In [22]:
## reshaping data
input_shape = time_period * sensors
x_train = x_train.reshape(x_train.shape[0], input_shape)
print("Input Shape: ", input_shape)
print("Input Data Shape: ", x_train.shape)

Input Shape:  240
Input Data Shape:  (20334, 240)


In [24]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

In [25]:
y_train_hot = to_categorical(y_train, num_classes)
print("y_train shape: ", y_train_hot.shape)

y_train shape:  (20334, 6)


In [26]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(input_shape,1), activation='relu'))
model.add(LSTM(32,return_sequences=True, activation='relu'))
model.add(Reshape((1, 240, 32)))
model.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
model.add(Reshape((120, 64)))
model.add(MaxPool1D(pool_size=4, padding='same'))
model.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
model.add(Reshape((29, 192)))
model.add(GlobalAveragePooling1D())
model.add(BatchNormalization(epsilon=1e-06))
model.add(Dense(6))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 240, 32)           4352      
                                                                 
 lstm_1 (LSTM)               (None, 240, 32)           8320      
                                                                 
 reshape (Reshape)           (None, 1, 240, 32)        0         
                                                                 
 conv1d (Conv1D)             (None, 1, 120, 64)        4160      
                                                                 
 reshape_1 (Reshape)         (None, 120, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 30, 64)            0         
 D)                                                              
                                                        

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train,
                    y_train_hot, 
                    batch_size= 192, 
                    epochs=100
                   )

Epoch 1/100
106/106 [==============================] - 40s 348ms/step - loss: 0.9026 - accuracy: 0.6951
Epoch 2/100
106/106 [==============================] - 39s 372ms/step - loss: 0.5679 - accuracy: 0.7910
Epoch 3/100
106/106 [==============================] - 39s 365ms/step - loss: 0.5057 - accuracy: 0.8120
Epoch 4/100
106/106 [==============================] - 37s 353ms/step - loss: 0.4643 - accuracy: 0.8303
Epoch 5/100
106/106 [==============================] - 37s 353ms/step - loss: 0.4239 - accuracy: 0.8420
Epoch 6/100
106/106 [==============================] - 37s 350ms/step - loss: 0.3833 - accuracy: 0.8565
Epoch 7/100
106/106 [==============================] - 40s 374ms/step - loss: 0.3438 - accuracy: 0.8686
Epoch 8/100
106/106 [==============================] - 39s 369ms/step - loss: 0.3211 - accuracy: 0.8798
Epoch 9/100
106/106 [==============================] - 38s 358ms/step - loss: 0.3118 - accuracy: 0.8838
Epoch 10/100
106/106 [==============================] - 46s 439m

106/106 [==============================] - 40s 373ms/step - loss: 0.1181 - accuracy: 0.9597
Epoch 80/100
106/106 [==============================] - 38s 354ms/step - loss: 0.1123 - accuracy: 0.9621
Epoch 81/100
106/106 [==============================] - 38s 354ms/step - loss: 0.1146 - accuracy: 0.9614
Epoch 82/100
106/106 [==============================] - 38s 355ms/step - loss: 0.1170 - accuracy: 0.9618
Epoch 83/100
106/106 [==============================] - 37s 352ms/step - loss: 0.1172 - accuracy: 0.9605
Epoch 84/100
106/106 [==============================] - 39s 364ms/step - loss: 0.1148 - accuracy: 0.9620
Epoch 85/100
106/106 [==============================] - 41s 389ms/step - loss: 0.1137 - accuracy: 0.9615
Epoch 86/100
106/106 [==============================] - 38s 355ms/step - loss: 0.1199 - accuracy: 0.9588
Epoch 87/100
106/106 [==============================] - 40s 374ms/step - loss: 0.1125 - accuracy: 0.9610
Epoch 88/100
106/106 [==============================] - 37s 352ms/st

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

y_pred_train = model.predict(x_train)
max_y_pred_train = np.argmax(y_pred_train, axis=1)
print(classification_report(y_train, max_y_pred_train))

In [ ]:
df_test['X'] = (df_test['X']-df_test['X'].min())/(df_test['X'].max()-df_test['X'].min())
df_test['Y'] = (df_test['Y']-df_test['Y'].min())/(df_test['Y'].max()-df_test['Y'].min())
df_test['Z'] = (df_test['Z']-df_test['Z'].min())/(df_test['Z'].max()-df_test['Z'].min())
x_test, y_test = segments(df_test,
                         TIME_PERIOD,
                         STEP_DISTANCE,
                         LABEL)

x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test = to_categorical(y_test, num_classes)

In [ ]:
score = model.evaluate(x_test, y_test)
print("Accuracy:", score[1])
print("Loss:", score[0])

In [ ]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=1)
y_test_pred = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test_pred, predictions)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()

In [ ]:
print(classification_report(y_test_pred, predictions))